Using the `all data` gdb of historical fires from Calfire:
https://www.fire.ca.gov/what-we-do/fire-resource-assessment-program/fire-perimeters

In [1]:
import requests
import zipfile
import os
import fiona
import geopandas as gpd
import json
from shapely.geometry import mapping

import sys
sys.path.append("..")

from src.routers.dependencies import get_cloud_logger, get_cloud_static_io_client 

from dotenv import load_dotenv
load_dotenv("../.devcontainer/.env")

from google.cloud import tasks_v2
from google.protobuf import timestamp_pb2
import geopandas as gpd
import datetime
import json
from shapely.geometry import box
from src.routers.batch.batch_analyze_and_fetch import main as batch_analyze_and_fetch
from google.cloud import logging
from src.util.cloud_static_io import CloudStaticIOClient

# Extract zip file
with zipfile.ZipFile("fire221gdb.zip", "r") as zip_ref:
    zip_ref.extractall(".")

# Get the name of the gdb file
gdb_file = [f for f in os.listdir(".") if f.endswith(".gdb")][0]

# List all layers in the GDB file
layers = fiona.listlayers(gdb_file)

In [2]:
%load_ext autoreload
%autoreload 2

According to calfire:

```
The fire perimeters database includes three layers—historical fire perimeters (firep), Rx treatments using fire (rxburn), and Rx treatments not using fire (Non_RXFire_Legacy).
```

In [3]:
layers

['rxburn22_1', 'firep22_1', 'Non_RXFire_Legacy13_2']

In [4]:
rx_treatments_with_fire = gpd.read_file(gdb_file, layer=layers[0])
rx_treatments_without_fire = gpd.read_file(gdb_file, layer=layers[2])
fire_perimeters = gpd.read_file(gdb_file, layer=layers[1])

rx_treatments_with_fire.to_crs(epsg=4326, inplace=True)
rx_treatments_without_fire.to_crs(epsg=4326, inplace=True)
fire_perimeters.to_crs(epsg=4326, inplace=True)

In [5]:
rx_treatments_with_fire.head()

,YEAR_,STATE,AGENCY,UNIT_ID,TREATMENT_ID,TREATMENT_NAME,START_DATE,END_DATE,TREATED_AC,GIS_ACRES,RX_CONSUM,PRE_CON_CLASS,POST_CON_CLASS,TREATMENT_TYPE,Shape_Length,Shape_Area,geometry
0,2020,CA,CDF,AEU,10509,Jan 31 2020 Broadcast,2020-01-31 00:00:00+00:00,2020-01-31T00:00:00+00:00,15.0,15.047042,NaN,NaN,NaN,1.0,1365.887529,60893.216906,"MULTIPOLYGON (((-120.52136 38.75171, -120.5212..."
1,2020,CA,PVT,AEU,10572,2020 Fuels Reduction,2020-02-13 00:00:00+00:00,2020-02-13T00:00:00+00:00,22.3,36.739700,NaN,NaN,NaN,1.0,2681.610336,148680.293398,"MULTIPOLYGON (((-120.66046 38.90069, -120.6605..."
2,2020,CA,CDF,AEU,10630,Feb 2020 Broadcast,2020-02-13 00:00:00+00:00,2020-02-18T00:00:00+00:00,22.4,38.839832,NaN,NaN,NaN,1.0,3918.219956,157179.228865,"MULTIPOLYGON (((-120.57195 38.50208, -120.5718..."
3,2020,CA,CDF,AEU,10639,Feb 2020 Broadcast,2020-02-11 00:00:00+00:00,2020-02-20T00:00:00+00:00,75.5,75.498100,NaN,NaN,NaN,1.0,3918.834462,305529.980211,"MULTIPOLYGON (((-120.54192 38.74314, -120.5419..."
4,2020,CA,CDF,AEU,10780,Mar 3 2020 Broadcast,2020-03-03 00:00:00+00:00,2020-03-03T00:00:00+00:00,61.8,61.771519,NaN,NaN,NaN,1.0,2769.034316,249980.461716,"MULTIPOLYGON (((-120.54811 38.74108, -120.5481..."


In [6]:
rx_treatments_without_fire.head()

,TREATMENT_ID,TREATMENT_NAME,TREATMENT_TYPE,UNIT_ID,AGENCY,TREATED_AC,GIS_ACRES,STATE,YEAR_,RX_CONSUM,PRE_CON_CLASS,POST_CON_CLASS,END_DATE,START_DATE,Shape_Length,Shape_Area,geometry
0,3277604,NON_WUI,13,SQF,USF,136.388878,136.388840,CA,2006,NaN,NaN,NaN,2006-04-15 00:00:00+00:00,NaT,7391.270964,5.519460e+05,"MULTIPOLYGON (((-118.50884 36.10266, -118.5086..."
1,3277608,NON_WUI,15,SQF,USF,292.127163,292.126648,CA,2006,NaN,NaN,NaN,2006-07-03 00:00:00+00:00,NaT,17166.848106,1.182195e+06,"MULTIPOLYGON (((-118.96349 36.78626, -118.9632..."
2,3277612,SOUTHRIDGE 1-1 CHIP,5,BDF,USF,4.726939,4.726954,CA,2006,NaN,NaN,NaN,2006-02-01 00:00:00+00:00,NaT,880.091816,1.912930e+04,"MULTIPOLYGON (((-116.69961 33.73683, -116.6997..."
3,3277613,SOUTHRIDGE UNIT 5 CUT AND STACK,9,BDF,USF,22.742776,22.742844,CA,2006,NaN,NaN,NaN,2006-03-01 00:00:00+00:00,NaT,1623.896026,9.203702e+04,"MULTIPOLYGON (((-116.70554 33.72558, -116.7052..."
4,3277840,HIGHWAY 20 CUT & PILING (06),9,TNF,USF,106.348339,106.347382,CA,2006,NaN,NaN,NaN,2006-09-15 00:00:00+00:00,NaT,3002.279795,4.303726e+05,"MULTIPOLYGON (((-120.79859 39.31701, -120.7983..."


In [7]:
fire_perimeters.head()

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,COMPLEX_NAME,COMPLEX_INCNUM,IRWINID,Shape_Length,Shape_Area,geometry
0,2020,CA,CDF,NEU,NELSON,00013212,2020-06-18T00:00:00+00:00,2020-06-23T00:00:00+00:00,11.0,,109.602280,1.0,1.0,None,None,None,None,3252.523280,4.435447e+05,"MULTIPOLYGON (((-121.34841 38.88999, -121.3483..."
1,2020,CA,CDF,NEU,AMORUSO,00011799,2020-06-01T00:00:00+00:00,2020-06-04T00:00:00+00:00,2.0,,685.585022,1.0,1.0,None,None,None,None,9653.760308,2.774464e+06,"MULTIPOLYGON (((-121.35275 38.82039, -121.3526..."
2,2020,CA,CDF,NEU,ATHENS,00018493,2020-08-10T00:00:00+00:00,2020-08-11T00:00:00+00:00,14.0,,27.300480,1.0,1.0,None,None,None,None,1649.643235,1.104811e+05,"MULTIPOLYGON (((-121.33334 38.84558, -121.3331..."
3,2020,CA,CDF,NEU,FLEMING,00007619,2020-03-31T00:00:00+00:00,2020-04-01T00:00:00+00:00,9.0,,12.931545,1.0,1.0,None,None,None,None,1577.155857,5.233211e+04,"MULTIPOLYGON (((-121.27317 38.96308, -121.2730..."
4,2020,CA,CDF,NEU,MELANESE,00008471,2020-04-14T00:00:00+00:00,2020-04-19T00:00:00+00:00,18.0,,10.315964,1.0,1.0,None,None,None,None,1035.787625,4.174722e+04,"MULTIPOLYGON (((-121.30066 39.48714, -121.3004..."


In [8]:
fire_perimeters[fire_perimeters["AGENCY"] == "NPS"]

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,COMPLEX_NAME,COMPLEX_INCNUM,IRWINID,Shape_Length,Shape_Area,geometry
276,2020,CA,NPS,MNP,BULL,00013423,2020-09-05T00:00:00+00:00,2020-09-05T00:00:00+00:00,10.0,AFC018BF-96ED-4018-AB90-77E625432EA7,14.089692,8.0,1.0,None,None,None,None,1655.085138,5.701896e+04,"MULTIPOLYGON (((-115.79681 35.40100, -115.7961..."
305,2020,CA,NPS,YNP,BLUEJAY,00000054,2020-07-25T00:00:00+00:00,2020-11-19T00:00:00+00:00,1.0,,6922.013672,7.0,2.0,None,None,None,None,44088.647246,2.801240e+07,"MULTIPOLYGON (((-119.60746 37.81324, -119.6074..."
306,2020,CA,NPS,BNP,CALDWELL,00000479,2020-07-22T00:00:00+00:00,2020-09-01T00:00:00+00:00,1.0,,81224.679688,7.0,1.0,None,None,None,None,176044.495381,3.287046e+08,"MULTIPOLYGON (((-121.60456 41.83603, -121.6045..."
307,2020,CA,NPS,MNP,DOME,00012356,2020-08-15T00:00:00+00:00,2020-09-12T00:00:00+00:00,1.0,Auto-generated by EGP-IRWIN,44211.250000,8.0,1.0,None,None,None,None,76882.528186,1.789166e+08,"MULTIPOLYGON (((-115.54279 35.36503, -115.5461..."
308,2020,CA,NPS,YNP,HORSE,00000089,2020-08-23T00:00:00+00:00,2020-11-19T00:00:00+00:00,1.0,,30.192495,1.0,2.0,None,None,None,None,4215.009845,1.221847e+05,"MULTIPOLYGON (((-119.53190 37.62689, -119.5319..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21856,2022,CA,NPS,YNP,RED,00000056,2022-08-04T00:00:00+00:00,2022-11-06T00:00:00+00:00,1.0,None,8432.418945,3.0,2.0,None,None,None,1E643C3B-3BE4-4669-892B-CF82B46FA820,98931.614579,3.412479e+07,"MULTIPOLYGON (((-119.48282 37.70949, -119.4828..."
21857,2022,CA,NPS,YNP,RODGERS,00000058,2022-08-08T00:00:00+00:00,2022-11-06T00:00:00+00:00,1.0,None,2839.684082,3.0,2.0,None,None,None,99B90DBC-6F1A-4494-B5C6-9CFDF335492A,54091.789770,1.149179e+07,"MULTIPOLYGON (((-119.52391 37.94608, -119.5239..."
21858,2022,CA,NPS,KNP,AVALANCHE,00000049,2022-07-18T00:00:00+00:00,2022-11-10T00:00:00+00:00,1.0,None,28.925026,7.0,2.0,None,None,None,52D1B9EC-9479-48E6-AF4A-D1550E8A97CB,1415.510211,1.170554e+05,"MULTIPOLYGON (((-118.58141 36.77579, -118.5815..."
21859,2022,CA,NPS,KNP,SUMMIT,00000075,2022-08-03T00:00:00+00:00,2022-10-19T00:00:00+00:00,1.0,None,1394.439575,7.0,2.0,None,None,None,F3705FA2-72E3-4E2A-BBE0-EA04D37FAE54,15630.965622,5.643097e+06,"MULTIPOLYGON (((-118.66634 36.33272, -118.6663..."


To batch process (some) of these, we can use our batch endpoint, which accepts all of the form arguments we get from the frontend and does our anaylze and fetch workflows. 

In [35]:
# Create a client
client = tasks_v2.CloudTasksClient()

# Get logger
# logging_client = logging.Client(project="dse-nps")
# log_name = "burn-backend"
# logger = logging_client.logger(log_name)
logger = get_cloud_logger()

# Get cloud static io client
# s3_bucket_name = os.getenv("S3_BUCKET_NAME")
# cloud_static_io_client = CloudStaticIOClient(s3_bucket_name, "s3")
cloud_static_io_client = get_cloud_static_io_client(logger=logger)

# Define our Google Cloud Project ID and Queue ID
project = "dse-nps"
queue = "tf-rest-burn-severity-queue-dev"
location = "us-central1"

# Define the URL endpoint of our cloud function
dev_url = "https://tf-rest-burn-severity-dev-ohi6r6qs2a-uc.a.run.app"
# dev_url = "localhost:5050"
batch_url = dev_url + "/api/batch/analyze-and-fetch"

/opt/conda/envs/burn-severity/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/burn-severity/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/burn-severity/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your applica

In [ ]:
import json
from shapely.geometry import mapping

# Convert DataFrame row to dictionary
def row_to_dict(row):
    return row.to_dict()


# Create a task with row data as payload
def create_task(row):
    # Convert the row to a dictionary
    row_dict = row_to_dict(row)

    coord_list = list(row['geometry'].boundary.convex_hull.exterior.coords)
    bounds = box(*row['geometry'].bounds)

    del row_dict['geometry']

    row_geojson = {
                    "type": "FeatureCollection",
                    "features": [
                        {
                            "id": "0",
                            "type": "Feature",
                            "properties": row_dict,
                            "geometry": {
                                "type": "Polygon",
                                "coordinates": [coord_list]
                            }
                        }
                    ]
                }

    ignition_date = datetime.datetime.strptime(
        row_dict["ALARM_DATE"], "%Y-%m-%dT%H:%M:%S%z"
    )
    containment_date = datetime.datetime.strptime(
        row_dict["CONT_DATE"], "%Y-%m-%dT%H:%M:%S%z"
    )

    post_body = {
        "fire_event_name": row_dict["FIRE_NAME"],
        "affiliation": row_dict["AGENCY"] + "_CalFire",
        "ignition_date": str(ignition_date),
        "containment_date": str(containment_date),
        "time_buffer_days": 30,
        "derive_boundary": False,
        "geojson_boundary": json.dumps(row_geojson),
    }

    # batch_analyze_and_fetch(
    #     geojson_boundary=post_body["geojson_boundary"],
    #     fire_event_name=post_body["fire_event_name"],
    #     affiliation=post_body["affiliation"],
    #     derive_boundary=post_body["derive_boundary"],
    #     logger=logger,
    #     cloud_static_io_client=cloud_static_io_client,
    #     ignition_date=ignition_date,
    #     containment_date=containment_date,
    #     time_buffer_days=post_body["time_buffer_days"],
    # )

    ### FIRST, the boundary as given by calfire
    payload = json.dumps(post_body).encode()

    # ## Hit the local endpoint

    # local_url = "http://localhost:5050/api/batch/analyze-and-fetch"
    # response = requests.post(local_url, data=payload, headers={"Content-type": "application/json"})
    # return response
    # Construct the request body
    parent = f"projects/{project}/locations/{location}/queues/{queue}"
    task = {
        "http_request": {
            "http_method": "POST",
            "url": batch_url,
            "body": payload,
            "headers": {"Content-type": "application/json"},
        }
    }

    # Add the timestamp to the tasks
    timestamp = timestamp_pb2.Timestamp()
    timestamp.FromDatetime(datetime.datetime.utcnow() + datetime.timedelta(seconds=10))
    task["schedule_time"] = timestamp

    # Use the client to build and send the task
    response = client.create_task(request={"parent": parent, "task": task})

    print("Created task {}".format(response.name))

    ### SECOND, the boundary as derived by the simple threshold
    post_body["derive_boundary"] = True
    post_body["fire_event_name"] = post_body["fire_event_name"] + "_derived"

    buffered_bbox = bounds.buffer(0.1)
    buffered_bbox_coords = mapping(buffered_bbox)["coordinates"]

    row_approx_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "id": "0",
                "type": "Feature",
                "properties": row_dict,
                "geometry": {"type": "Polygon", "coordinates": buffered_bbox_coords},
            }
        ],
    }
    post_body["geojson_boundary"] = json.dumps(row_approx_geojson)

    payload = json.dumps(post_body).encode()

    # Construct the request body
    parent = f"projects/{project}/locations/{location}/queues/{queue}"
    task = {
        "http_request": {
            "http_method": "POST",
            "url": batch_url,
            "body": payload,
            "headers": {"Content-type": "application/json"},
        }
    }

    # Add the timestamp to the tasks
    timestamp = timestamp_pb2.Timestamp()
    timestamp.FromDatetime(datetime.datetime.utcnow() + datetime.timedelta(seconds=10))
    task["schedule_time"] = timestamp

    # Use the client to build and send the task
    response = client.create_task(request={"parent": parent, "task": task})

    print("Created task {}".format(response.name))

For our test, we want Southern CA fires during Sentinel 2's time:

In [11]:
lat_min = 32
lon_min = -120
lat_max = 37
lon_max = -115

# make into a polygon geopandas for sjoin
bbox = box(lon_min, lat_min, lon_max, lat_max)
bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")
bbox_gdf

,geometry
0,"POLYGON ((-115.00000 32.00000, -115.00000 37.0..."


In [12]:
bbox.bounds

(-120.0, 32.0, -115.0, 37.0)

In [13]:
from geopandas.tools import sjoin
sample_fire_perimeter = sjoin(fire_perimeters, bbox_gdf, how="inner")
sample_fire_perimeter = sample_fire_perimeter[sample_fire_perimeter["YEAR_"] != '']
sample_fire_perimeter["YEAR_"] = sample_fire_perimeter["YEAR_"].astype("int")
nps_fires = sample_fire_perimeter[(sample_fire_perimeter["YEAR_"] > 2015) & (sample_fire_perimeter["AGENCY"] == "NPS")]

In [14]:
nps_fires

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,...,C_METHOD,OBJECTIVE,FIRE_NUM,COMPLEX_NAME,COMPLEX_INCNUM,IRWINID,Shape_Length,Shape_Area,geometry,index_right
276,2020,CA,NPS,MNP,BULL,00013423,2020-09-05T00:00:00+00:00,2020-09-05T00:00:00+00:00,10.0,AFC018BF-96ED-4018-AB90-77E625432EA7,...,8.0,1.0,None,None,None,None,1655.085138,5.701896e+04,"MULTIPOLYGON (((-115.79681 35.40100, -115.7961...",0
307,2020,CA,NPS,MNP,DOME,00012356,2020-08-15T00:00:00+00:00,2020-09-12T00:00:00+00:00,1.0,Auto-generated by EGP-IRWIN,...,8.0,1.0,None,None,None,None,76882.528186,1.789166e+08,"MULTIPOLYGON (((-115.54279 35.36503, -115.5461...",0
309,2020,CA,NPS,MNP,IVANPAH,00009202,2020-06-22T00:00:00+00:00,2020-06-26T00:00:00+00:00,8.0,Auto-generated by EGP-IRWIN,...,8.0,1.0,None,None,None,None,10036.750481,4.255211e+06,"MULTIPOLYGON (((-115.17324 35.19340, -115.1734...",0
310,2020,CA,NPS,KNP,MORAINE,00000091,2020-08-21T00:00:00+00:00,2020-12-29T00:00:00+00:00,1.0,,...,2.0,1.0,None,None,None,None,12433.665515,5.325816e+06,"MULTIPOLYGON (((-118.56183 36.72931, -118.5617...",0
312,2020,CA,NPS,KNP,RATTLESNAKE,00000080,2020-08-16T00:00:00+00:00,2020-12-28T00:00:00+00:00,1.0,estimate based on satellite heat since 21 NOV,...,7.0,2.0,None,None,None,None,114391.760771,3.408614e+07,"MULTIPOLYGON (((-118.40697 36.36010, -118.4073...",0
313,2020,CA,NPS,CNP,SCORPION,00001216,2020-05-31T00:00:00+00:00,2020-06-05T00:00:00+00:00,2.0,,...,1.0,1.0,None,None,None,None,23541.766224,5.344798e+06,"MULTIPOLYGON (((-119.55486 34.04815, -119.5538...",0
12374,2016,CA,NPS,SMP,COAST,000896,2016-12-02T00:00:00+00:00,2016-12-02T00:00:00+00:00,14.0,None,...,8.0,1.0,None,None,None,None,461.384327,4.724250e+03,"MULTIPOLYGON (((-118.94804 34.04749, -118.9480...",0
12375,2016,CA,NPS,SMP,LIBERTY CANYON,000843,2016-11-05T00:00:00+00:00,2016-11-05T00:00:00+00:00,14.0,None,...,8.0,1.0,None,None,None,None,63.357001,3.145807e+02,"MULTIPOLYGON (((-118.72723 34.13779, -118.7272...",0
12376,2016,CA,NPS,SMP,MULHOLLAND,000210,2016-02-25T00:00:00+00:00,2016-02-28T00:00:00+00:00,14.0,LA Co. INC # 058094,...,8.0,1.0,None,None,None,None,1032.685562,4.583182e+04,"MULTIPOLYGON (((-118.91151 34.08538, -118.9115...",0
12377,2016,CA,NPS,KNP,YUCCA,000184,2016-11-05T00:00:00+00:00,2016-11-15T00:00:00+00:00,1.0,Yucca 2016,...,8.0,1.0,None,None,None,None,4271.848972,2.901110e+05,"MULTIPOLYGON (((-118.85826 36.60500, -118.8583...",0


In [55]:
for index, row in nps_fires.iterrows():
    create_task(row)


Created task projects/dse-nps/locations/us-central1/queues/tf-rest-burn-severity-queue-dev/tasks/84070019243162664341
